In [ ]:
import pandas as pd
from google.colab import files
import io

uploaded = files.upload()

Saving full_leafly_merge.csv to full_leafly_merge.csv


In [ ]:
#this is the second stage of cleaning/merging

In [ ]:
med = pd.read_csv(io.BytesIO(uploaded['merged.csv']))

#this is the data from our second med build from kaggle merged with updated info from same kaggle data

In [ ]:
leafly_merge = pd.read_csv(io.BytesIO(uploaded['full_leafly_merge.csv']))

#this is data from the otreeba API merged with data scraped from leafly

In [ ]:
print('Med cabinet data shape:', med.shape, 'Leafly and api merged shape:', leafly_merge.shape)

Med cabinet data shape: (2826, 9) Leafly and api merged shape: (10796, 15)


In [ ]:
med.head()

,id,name,race,flavors,positive,negative,medical,Rating,Description
0,1,Afpak,hybrid,"['Earthy', 'Chemical', 'Pine', 'Spicy/Herbal']","['Relaxed', 'Hungry', 'Happy', 'Sleepy', 'Crea...",['Dizzy'],"['Depression', 'Insomnia', 'Pain', 'Stress', '...",4.2,"Afpak, named for its direct Afghani and Pakist..."
1,2,African,sativa,"['Spicy/Herbal', 'Pungent', 'Earthy', 'Pepper']","['Euphoric', 'Happy', 'Creative', 'Energetic',...",['Dry Mouth'],"['Depression', 'Pain', 'Stress', 'Lack of Appe...",3.9,African refers to the indigenous varieties of ...
2,3,Afternoon Delight,hybrid,"['Pepper', 'Flowery', 'Pine', 'Pungent', 'Citr...","['Relaxed', 'Hungry', 'Euphoric', 'Uplifted', ...","['Dizzy', 'Dry Mouth', 'Paranoid']","['Depression', 'Insomnia', 'Pain', 'Stress', '...",4.8,"Afternoon Delight, created by Colorado Seed In..."
3,4,Afwreck,hybrid,"['Pine', 'Earthy', 'Flowery', 'Pungent']","['Relaxed', 'Happy', 'Creative', 'Uplifted', '...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Pain', 'Stress', 'Headache', 'Fatigue', 'Hea...",4.2,Afwreck is a hybrid cross of Afghani and Train...
4,5,Agent Orange,hybrid,"['Citrus', 'Orange', 'Sweet', 'Earthy']","['Relaxed', 'Euphoric', 'Happy', 'Energetic', ...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes']","['Depression', 'Pain', 'Stress', 'Nausea', 'He...",4.2,Don’t let the name scare you! The only herbici...


In [ ]:
med = med.rename(columns={'Description': 'description'})
med = med.drop('Rating', axis = 1)

In [ ]:
med.columns

Index(['id', 'name', 'race', 'flavors', 'positive', 'negative', 'medical',
       'description'],
      dtype='object')

In [ ]:
def clean_names(x):
    '''it is easier to clean the names from the href than it is to scrape them'''
    if 'strains' in x:
        x = x.replace('strains', ',')
    x = x.replace('/', '')
    x = x.replace('-', ' ')
    x = x.replace(',', ', ')
    x = x.replace('#', '')
    x = x.title()
    if x[0] ==',':
        x = x[1:]
    if x[0] ==' ':
        x = x[1:]
    return x

In [ ]:
med['name'] = med['name'].apply(clean_names)

In [ ]:
leafly_merge.head()

,Unnamed: 0,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews
0,0,1K,NaN,None,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1K PC,NaN,1K x Purple Caper,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,32,NaN,Albert Walker x Manic,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,32 x Face Off OG Kush,NaN,32 x Faceoff Cut OG Kush,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,78 LA OG Affie,NaN,Lemon Thai x Colombian Gold x Pakistani Landrace,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
counts_samename = 0
strains_in_both = []
for i in leafly_merge['name']:
  for x in med['name']:
    if x == i:
      counts_samename += 1
      strains_in_both += ([i])
counts_samename

2630

In [ ]:
leafly_med_merge = pd.merge(leafly_merge, med, how='outer', on='name')

In [ ]:
#get rid of duplicates
leafly_med_merge = leafly_med_merge.drop_duplicates(subset=['name'], keep='first')

In [ ]:
leafly_med_merge.columns

Index(['Unnamed: 0', 'name', 'image', 'genetics', 'lineage', 'description_x',
       'terpene_description', 'technical_terpenes', 'parents', 'children',
       'reported_feelings', 'pic', 'thc_content', 'rating', 'reviews', 'id',
       'race', 'flavors', 'positive', 'negative', 'medical', 'description_y'],
      dtype='object')

In [ ]:
leafly_med_merge = leafly_med_merge.drop(['Unnamed: 0', 'id', 'description_y'], axis=1)

In [ ]:
leafly_med_merge = leafly_med_merge.rename(columns={'description_x': 'description'})

In [ ]:
leafly_med_merge.head()

,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews,race,flavors,positive,negative,medical
0,1K,NaN,None,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1K PC,NaN,1K x Purple Caper,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,NaN,Albert Walker x Manic,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32 x Face Off OG Kush,NaN,32 x Faceoff Cut OG Kush,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,78 LA OG Affie,NaN,Lemon Thai x Colombian Gold x Pakistani Landrace,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cols = list(leafly_med_merge.columns)
cols

['name',
 'image',
 'genetics',
 'lineage',
 'description',
 'terpene_description',
 'technical_terpenes',
 'parents',
 'children',
 'reported_feelings',
 'pic',
 'thc_content',
 'rating',
 'reviews',
 'race',
 'flavors',
 'positive',
 'negative',
 'medical']

In [ ]:
for i in cols:
  null = leafly_med_merge[i].isnull().sum()
  none = (leafly_med_merge[i] == 'None').sum()
  brackets = (leafly_med_merge[i] == '[]').sum()
  print(i,'---', 'null:', null, 'none:', none, 'brackets:', brackets)

name --- null: 0 none: 0 brackets: 0
image --- null: 7218 none: 0 brackets: 0
genetics --- null: 2292 none: 1934 brackets: 0
lineage --- null: 2292 none: 0 brackets: 2610
description --- null: 7617 none: 0 brackets: 0
terpene_description --- null: 7581 none: 0 brackets: 3014
technical_terpenes --- null: 10595 none: 0 brackets: 0
parents --- null: 7581 none: 0 brackets: 401
children --- null: 7581 none: 0 brackets: 2936
reported_feelings --- null: 7581 none: 0 brackets: 470
pic --- null: 7581 none: 0 brackets: 0
thc_content --- null: 8425 none: 0 brackets: 1525
rating --- null: 7581 none: 315 brackets: 0
reviews --- null: 7581 none: 315 brackets: 0
race --- null: 8440 none: 0 brackets: 0
flavors --- null: 8440 none: 0 brackets: 581
positive --- null: 8440 none: 0 brackets: 581
negative --- null: 8440 none: 0 brackets: 969
medical --- null: 8440 none: 0 brackets: 635


In [ ]:
#replace all the none and [] values with nans for dropna to be more accurate
for i in cols:
  leafly_med_merge[i] = leafly_med_merge[i].replace(['None', '[]', ' '], float('nan'))

In [ ]:
leafly_med_merge.head(10)

,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews,race,flavors,positive,negative,medical
0,1K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1K PC,NaN,1K x Purple Caper,Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,NaN,Albert Walker x Manic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32 x Face Off OG Kush,NaN,32 x Faceoff Cut OG Kush,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,78 LA OG Affie,NaN,Lemon Thai x Colombian Gold x Pakistani Landrace,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,86 Black UW,NaN,U Dub (UW),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,(Ducksfoot x Lowryder) x AK 47,https://www.cannabisreports.com/images/strains...,( Ducksfoot x Lowryder ) x AK 47,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,(Early Pearl/Skunk No. 1/Northern Lights 5) x ...,NaN,(Early Pearl / Skunk No. 1 / Northern Lights 5...,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,(G 13 x Blue Widow) x Killer Queen,NaN,"""(Airbornes G13 x Blue Widow) x Killer Queen""",Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,(Hash Plant / Northern Lights 1 / ) x Swaziland,NaN,(Hash Plant (Sensi) / Northern Lights 1 / ) x ...,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
thresh = 4

test2 = leafly_med_merge.dropna(axis=0, subset=['image','lineage','description','terpene_description', 'thc_content',
                                               'parents','reported_feelings','race','flavors','positive'], thresh=thresh)

print('Before dropping rows with over', thresh, 'missing:', leafly_med_merge.shape, 'After:', test2.shape)

Before dropping rows with over 4 missing: (10991, 19) After: (2583, 19)


In [ ]:
test2.head(50)

,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews,race,flavors,positive,negative,medical
16,1024,https://www.cannabisreports.com/images/strains...,NaN,NaN,1024 is a sativa-dominant hybrid bred in Spain...,NaN,NaN,NaN,Aloha Grape Stomper,"['Creative', 'Focused', 'Euphoric', 'Happy', '...",https://public.leafly.com/strains/flowers/1024...,NaN,4.5,55 reviews,sativa,NaN,NaN,NaN,NaN
19,13 Dawgs,NaN,G 13 (G13) x Chem Dawg (Chemdog),NaN,13 Dawgs is a hybrid of G13 and Chemdawg genet...,NaN,NaN,"G 13, Chemdawg",NaN,"['Euphoric', 'Sleepy', 'Creative', 'Hungry', '...",https://public.leafly.com/strains/flowers/13-d...,NaN,4.1,12 reviews,hybrid,NaN,NaN,NaN,NaN
23,24K Gold,NaN,Kosher Kush x Tangie,NaN,"Also known as Kosher Tangie, 24k Gold is a 60%...","['Herbal', 'Pine', 'Citrus']","['{'terpine': '170%', 'name': 'Myrcene'}', '{...","Kosher Kush, Tangie",NaN,"['Happy', 'Relaxed', 'Relaxed', 'Euphoric', 'R...",https://public.leafly.com/strains/flowers/24k-...,20.5%,4.6,260 reviews,hybrid,NaN,NaN,NaN,NaN
25,3 Kings,https://www.cannabisreports.com/images/strains...,Headband / OG Kush / Sour Diesel,Afghanistan,"The 3 Kings marijuana strain, a holy trinity o...","['Peppery', 'Citrus', 'Hoppy']","['{'terpine': '170%', 'name': 'Caryophyllene'...","Headband, Og Kush",NaN,"['Creative', 'Energetic', 'Focused', 'Uplifted...",https://public.leafly.com/strains/flowers/3-ki...,18.5%,4.4,341 reviews,hybrid,NaN,NaN,NaN,NaN
47,5K Puff,https://www.cannabisreports.com/images/strains...,Hola Lightning x Joseph OG,Afghanistan,With the goal of seeing just how frosty a stra...,NaN,NaN,Triangle Kush,NaN,NaN,https://public.leafly.com/strains/flowers/5k-p...,22%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,707 Headband,https://www.cannabisreports.com/images/strains...,Headband,Afghanistan,"This strain emerged from the Humboldt County, ...","['Herbal', 'Peppery', 'Citrus']","['{'terpine': '170%', 'name': 'Myrcene'}', '{...","Master Kush, Og Kush","Madhatter, Bandana","['Euphoric', 'Happy', 'Hungry', 'Relaxed', 'Sl...",https://public.leafly.com/strains/flowers/707-...,18%,4.3,398 reviews,hybrid,NaN,NaN,NaN,NaN
57,707 Truthband,NaN,707 Headband x The Truth,Afghanistan,Crossing two carefully selected True OG phenot...,NaN,NaN,True Og,NaN,"['Creative', 'Euphoric', 'Focused', 'Happy', '...",https://public.leafly.com/strains/flowers/707-...,18%,4.5,2 reviews,NaN,NaN,NaN,NaN,NaN
62,8 Ball Kush,https://www.cannabisreports.com/images/strains...,Afghani Landrace / Pakistani Landrace,Afghanistan,"Bred by Barney’s Farm, 8 Ball Kush is a pure i...",NaN,NaN,Afghani,NaN,"['Creative', 'Euphoric', 'Giggly', 'Happy', 'R...",https://public.leafly.com/strains/flowers/8-ba...,NaN,4.2,60 reviews,indica,NaN,NaN,NaN,NaN
71,9 Pound Hammer,https://www.cannabisreports.com/images/strains...,GooBerry x Hells Angel OG x Jack The Ripper,Afghanistan,9 Pound Hammer is an indica created by JinxPro...,"['Herbal', 'Pine', 'Peppery']","['{'terpine': '170%', 'name': 'Myrcene'}', '{...","Gooberry, Hells Og","Safety Meeting, Shangri La","['Happy', 'Relaxed', 'Relaxed', 'Sleepy', 'Rel...",https://s3-us-west-2.amazonaws.com/leafly-imag...,19%,4.6,858 reviews,indica,NaN,NaN,NaN,NaN
81,A Dub,NaN,Sour Dubble x Alien Dog (Ether),Afghanistan,"Coming from the ever-popular Sour Diesel, OG K...",NaN,NaN,"Sour Bubble, Alien Dawg",NaN,"['Energetic', 'Euphoric', 'Happy', 'Relaxed', ...",https://public.leafly.com/strains/flowers/a-du...,25.5%,4.5,68 reviews,hybrid,NaN,NaN,NaN,NaN


In [ ]:
test2.to_csv(r'‪2_leafly_merge_subset.csv')

### Just some extra stuff I gotta do with this data don't mind me

In [ ]:
test2.loc[test2['technical_terpenes'] == "['width:170%', 'Myrcene', 'width:114%', 'Caryophyllene', 'width:58%', 'Linalool']"]

,name,image,genetics,lineage,description,terpene_description,technical_terpenes,parents,children,reported_feelings,pic,thc_content,rating,reviews,race,flavors,positive,negative,medical
9579,Goji Og,NaN,NaN,NaN,Goji OG is a sativa-dominant hybrid bred by Bo...,"['Herbal', 'Peppery', 'Floral']","['width:170%', 'Myrcene', 'width:114%', 'Caryo...","Snow Lotus, Nepalese","Alien Breath, Black Raspberry","['Euphoric', 'Euphoric', 'Relaxed', 'Euphoric'...",https://s3-us-west-2.amazonaws.com/leafly-imag...,19.5%,4.5,243 reviews,hybrid,"['Berry', 'Sweet', 'Earthy', '']","['Relaxed', 'Euphoric', 'Happy', 'Energetic', ...","['Dizzy', 'Dry Mouth', 'Paranoid', 'Dry Eyes',...","['Depression', 'Insomnia', 'Pain', 'Stress', '..."


In [ ]:
terp_dictionary = {'Myrcene':'Herbal', 'Pinene':'Pine', 'Caryophyllene':'Peppery',
                   'Limonene':'Citrus', 'Humulene':'Hoppy', 'Terpinolene':'Fruity', 
                   'Ocimene': 'Minty', 'Linalool':'Floral'}

In [ ]:
# I need to make a dictionary of terp description to the real terp
test2['technical_terpenes'].value_counts()

['width:170%', 'Myrcene', 'width:114%', 'Pinene', 'width:58%', 'Caryophyllene']          49
['width:170%', 'Myrcene', 'width:114%', 'Caryophyllene', 'width:58%', 'Limonene']        46
['width:170%', 'Caryophyllene', 'width:114%', 'Limonene', 'width:58%', 'Myrcene']        39
['width:170%', 'Myrcene', 'width:114%', 'Limonene', 'width:58%', 'Caryophyllene']        39
['width:170%', 'Myrcene', 'width:114%', 'Caryophyllene', 'width:58%', 'Pinene']          28
['width:170%', 'Caryophyllene', 'width:114%', 'Limonene', 'width:58%', 'Humulene']       28
['width:170%', 'Limonene', 'width:114%', 'Caryophyllene', 'width:58%', 'Myrcene']        25
['width:170%', 'Limonene', 'width:114%', 'Myrcene', 'width:58%', 'Caryophyllene']        17
['width:170%', 'Caryophyllene', 'width:114%', 'Myrcene', 'width:58%', 'Limonene']        16
['width:170%', 'Terpinolene', 'width:114%', 'Caryophyllene', 'width:58%', 'Pinene']       9
['width:170%', 'Terpinolene', 'width:114%', 'Myrcene', 'width:58%', 'Caryophylle